# After the cleaning in cleaning file i will join the files togither

0   &emsp; 5059 <br>
2    &emsp; 394<br>
4    &emsp; 350<br>
1    &emsp; 282<br>
5   &emsp;  261<br>
3  &emsp;   184<br>
6   &emsp;  117<br>
7   &emsp;  115<br>
<br>
each day indicate how many apartment sold 0 means not sold <br>
1 means sold in day 7/8/2022<br>
2 means sold in day 8/8/2022 <br>
...         ...     ...<br>
7 means sold in day 12/8/2022<br>
<br>
i would like to say that i have the data in 1st , 2nd and 3rd of this month but i don't have the data in the 4th and 5th of this month so i started in the 6th 
<br>
1/8  i have <br>
2/8  i have <br>
3/8  i have <br>
4/8  i don't have <br>
5/8  i don't have <br>
6/8  i have <br>
7/8  i have <br>
8/8  i have <br>
9/8  i have <br>
10/8 i have <br>
11/8 i have <br>
12/8 i have  <br>
13/8 i have  <br>
...

In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.compose import TransformedTargetRegressor
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
import datetime
from datetime import timedelta
import ast
import json    
import missingno as msno

In [2]:
yest =pd.read_csv('../data/3-8-2022.csv') # the main which is 6-8-2022 but for the sake of the example i put 3-8 
Today =  pd.read_csv('../data/23-8-2022.csv') #the new day file this is just EX i stopped at 21-8-2022

In [3]:
Today.columns

Index(['user_id', 'id', 'title', 'price', 'content', 'imgs', 'refresh', 'beds',
       'livings', 'wc', 'area', 'street_width', 'age', 'last_update',
       'ketchen', 'ac', 'furnished', 'location', 'path', 'district', 'width',
       'length', 'advertiser_type', 'create_time', 'review', 'profileImg',
       'UserName', 'iam_verified'],
      dtype='object')

In [4]:
# yest.DayOfRent.value_counts()

manually i had to choose the day this day indicate at which day the apratment was rented 

1 means one day after i started web scraping 2 means the two days after i started ... i stopped at 15 (from 6th till 21st)

In [5]:
day = 1 #this the day of the file from the 6th

### create new column that is called DayOfRent zero indicate it's on market and any other number indicate at which day it was rented

In [6]:
yest['DayOfRent'] = 0 #we used this for the first time only to create the column

In [7]:
# yest[yest['DayOfRent'] == 0 ].shape

# join 

In [8]:
df3 = yest.merge(Today, on='id', how='left', indicator=True) #left join that will contain the data in the main file and the data in both (in both means not sold yet)(right means pulish in after 6th)


In [9]:
df3['_merge'].value_counts()

left_only     3538
both          3330
right_only       0
Name: _merge, dtype: int64

### there are 3 possibilities 

1- both  : still on market 

2- left_only : it got off market 

3- right_only : new apartments but we chose left merge so we only going to have the left and the inner 

## chooe only left 

and we incluse only zero (ON market ) if we didn't put  `(df3['DayOfRent'] == 0 )` it will show the off market in previous days
`

In [13]:
df3 = df3.loc[(df3['_merge'] == 'left_only') & (df3['DayOfRent'] == 0 )] 


In [14]:
df3.shape[1] 

33

after the merge we got double the amount  of feature remove the duplicates by :

In [15]:
# this will filter if it is both(inner ) so now i have only the left and for df3['sold'] == 0 this becouse if ot is 1 then this mean i checked it bedore and it is sold in 1 
df3.drop(df3.filter(regex='_y$').columns, axis=1, inplace=True) # delete the extra columns 

In [16]:
df3.shape

(3538, 33)

## these are the rented apartment from 3-8-2022 untill 8/23/2022 flag them with `day`

In [17]:
df3['DayOfRent'] = day #this will fill all the data above with the day of it 

In [18]:
df3['DayOfRent']

1       1
3       1
4       1
5       1
7       1
       ..
6857    1
6862    1
6863    1
6865    1
6867    1
Name: DayOfRent, Length: 3538, dtype: int64

 #  fill the original data with day if it is equal 

In [19]:
for i in list(df3['id']) : 
    yest.loc[yest["id"] == i, "DayOfRent"] = day #now we fill the original data with day


In [21]:
yest['DayOfRent'].value_counts()

1    3538
0    3330
Name: DayOfRent, dtype: int64

In [23]:
yest.to_csv(f'../data/AQAR_TEST.csv' , index=False)

# next timeBeforeRent